In [1]:
import pandas as pd 
from langchain.vectorstores import FAISS
import re


# from langchain.text_splitter import MarkdownHeaderTextSplitter
# import matplotlib.pyplot as plt
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
df = pd.read_csv("./data/book_data.csv")

In [3]:
df["book_desc"] = df["book_desc"].fillna('')

In [4]:
docstring = ' <s><s> '.join(df["book_desc"].values[:200])

In [5]:
# docstring = ''
# for string in df["book_desc"]:
#     docstring += string + " " 

In [6]:
def document_processing(data: str) -> str:
    return re.sub(r'[\r\n\t\f\v]+', '', data)

In [7]:
document = document_processing(docstring)

In [8]:
source_chunks = []
# splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)

for chunk in document.split(" <s><s> "):
    source_chunks.append(Document(page_content=chunk, metadata={}))

# всего получилось чанков:
len(source_chunks)

200

In [9]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('intfloat/multilingual-e5-small')

sentences = ["привет мир", "hello world", "здравствуй вселенная"]
embeddings = model.encode(sentences)
print(util.dot_score(embeddings, embeddings))

c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([[1.0000, 0.9085, 0.9092],
        [0.9085, 1.0000, 0.8981],
        [0.9092, 0.8981, 1.0000]])


In [10]:
embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-small',  # distilbert-base-uncased  sergeyzh/rubert-tiny-turbo
    multi_process=True,
    encode_kwargs={"normalize_embeddings": True}
)


c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\S\PycharmProjects\SimilarBooksRecommendation\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [12]:
def get_simillar_bool(book_desc: str, index_db: FAISS, n_chunks: int = 5) -> str:
    # Поиск релевантных отрезков из базы знаний
    docs = index_db.similarity_search(book_desc, k=n_chunks)

    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))

    return message_content

In [13]:
book_desc = df["book_desc"].values[0]

In [15]:
message_content = get_simillar_bool(book_desc=book_desc, index_db=db, n_chunks=5)

In [16]:
def print_formatted_message(message):
    lines = message.split("\n")
    for line in lines:
        if line.startswith("Отрывок документа №"):
            print(f"\n{line}\n{'=' * (len(line) - 1)}")
        else:
            print(line.strip())

In [17]:
print_formatted_message(message_content)



Отрывок документа №1
=====================Winning will make you famous. Losing means certain death.The nation of Panem, formed from a post-apocalyptic North America, is a country that consists of a wealthy Capitol region surrounded by 12 poorer districts. Early in its history, a rebellion led by a 13th district against the Capitol resulted in its destruction and the creation of an annual televised event known as the Hunger Games. In punishment, and as a reminder of the power and grace of the Capitol, each district must yield one boy and one girl between the ages of 12 and 18 through a lottery system to participate in the games. The 'tributes' are chosen during the annual Reaping and are forced to fight to the death, leaving only one survivor to claim victory.When 16-year-old Katniss's young sister, Prim, is selected as District 12's female representative, Katniss volunteers to take her place. She and her male counterpart Peeta, are pitted against bigger, stronger representatives, som

In [18]:
pd.DataFrame(df.iloc[0]).transpose()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...


In [19]:
desc_1 = "Anna is not sick, but she might as well be"
desc_2 = "In Beatrice Prior's dystopian Chicago world"
desc_3 = "Be prepared to meet three unforgettable women:Twenty-two-year-old"

In [20]:
df[df['book_desc'].apply(lambda x: desc_1 in x)]

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
67,Jodi Picoult,"Anna is not sick, but she might as well be. By...",NaN,Paperback,9.78074E+12,423 pages,4.06,955234,31913,My Sister's Keeper,Fiction|Contemporary|Womens Fiction|Chick Lit,https://images.gr-assets.com/books/1369504683l...


In [21]:
df[df['book_desc'].apply(lambda x: desc_2 in x)]

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
19,Veronica Roth,"In Beatrice Prior's dystopian Chicago world, s...",NaN,Paperback,9.78006E+12,487 pages,4.22,2493519,104329,Divergent,Young Adult|Science Fiction|Dystopia|Fiction|F...,https://images.gr-assets.com/books/1328559506l...


In [22]:
df[df['book_desc'].apply(lambda x: desc_3 in x)]

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
28,Kathryn Stockett,Be prepared to meet three unforgettable women:...,NaN,Hardcover,9.7804E+12,465 pages,4.46,1790905,80741,The Help,Fiction|Historical|Historical Fiction|Historic...,https://images.gr-assets.com/books/1346100365l...


In [23]:
pd.DataFrame(df.iloc[0]).transpose()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
